# Датасет после небольшой обработки

In [1]:
import os
import numpy as np
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

os.chdir('D:\github_data')

commit_data_dir = 'preprocessed_data_short'
os.makedirs(commit_data_dir, exist_ok=True)

## Объединим данные со всех репозиториев в один DataFrame

In [2]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format


dfs = []
large_files = []

for filename in tqdm(os.listdir(commit_data_dir)):
    try:
        df = pd.read_json(os.path.join(commit_data_dir, filename), orient='records')
        df['repo'] = filename.split('.json')[0].replace('_', '/')
        dfs.append(df)
    except ValueError:
        print('ValueError with', filename)
        large_files.append(filename)
    except MemoryError:
        print('MemoryError with', filename)
        large_files.append(filename)

100%|████████████████████████████████████████████████████████████████████████████████| 995/995 [02:14<00:00,  7.40it/s]


In [3]:
df = pd.concat(dfs, ignore_index=True, axis=0)
df.author = df.author.apply(lambda x: tuple(x))
df.head()

,author,date,message,diff,num_mods,diff_len,message_len,repo
0,"(Arnaud Barisain Monrose, DREAMTEAM69@gmail.com)",2010-06-24 22:59:02,Fixed 2 . 1 compatibility,[<FILE> . DS _ Store <nl> Binary files a / . D...,12,1106,5,abarisain/dmix
1,"(Arnaud Barisain Monrose, DREAMTEAM69@gmail.com)",2010-06-25 00:20:53,Cleaned up debugging things,[<FILE> MPDroid / src / com / arkanta / mpdroi...,1,197,4,abarisain/dmix
2,"(dreamteam69, dreamteam69@gmail.com)",2010-08-07 03:14:53,Updated project so it uses 2 . 2 apis,[<FILE> MPDroid / AndroidManifest . xml <nl> <...,3,314,9,abarisain/dmix
3,"(dreamteam69, dreamteam69@gmail.com)",2010-08-07 03:16:28,Fixed text shadow size and cover art DPI,[<FILE> MPDroid / res / layout / main . xml <n...,2,1466,8,abarisain/dmix
4,"(dreamteam69, dreamteam69@gmail.com)",2010-08-07 14:50:35,Fixed 2 . 1 compatibility,[<FILE> MPDroid / default . properties <nl> # ...,3,1634,5,abarisain/dmix


In [4]:
len(df)

1598723

## Посмотрим статистику по данным

### Статистика по коммитам для каждого автора

In [5]:
author_df = df.groupby(by='author')['date'].count()
author_df.describe()

count   57697.00
mean       27.71
std       161.89
min         1.00
25%         1.00
50%         2.00
75%         7.00
max     11190.00
Name: date, dtype: float64

Всего есть ~58k уникальных авторов, причем тех, у кого всего один коммит, достаточно много. Посмотрим на конкретное соотношение:

In [6]:
print(f'% of authors with only 1 commit: {100 * len(author_df.loc[author_df == 1]) / len(author_df):.2f}')

% of authors with only 1 commit: 41.52


In [7]:
print(len(df) - len(df.loc[df['author'].isin(author_df.index[author_df > 1].tolist())]))

23958


### Статистика по количеству токенов в диффах

Здесь токенизация проводится просто разделением по пробелам, и в данной версии датасета отброшены все примеры более чем с 2048 токенами.

In [8]:
df['diff_len'].describe()

count   1598723.00
mean        652.10
std         524.89
min          73.00
25%         216.00
50%         478.00
75%         977.00
max        2048.00
Name: diff_len, dtype: float64

In [9]:
print(f"% of examples with # of tokens in diff <= 512: {100 * len(df.loc[df['diff_len'] <= 512]) / len(df):.2f}")

% of examples with # of tokens in diff <= 512: 52.38


### Статистика по количеству токенов в сообщениях

In [10]:
df['message_len'].describe()

count   1598723.00
mean         18.62
std          20.50
min           4.00
25%           7.00
50%          11.00
75%          21.00
max         152.00
Name: message_len, dtype: float64

### Немного статистики по структуре диффов

При сборе данных учитывались только коммиты, в которых есть .java файлы, но помимо них вполне могут быть и другие. Посмотрим, какую долю от всех изменений составляют изменения .java-файлов: 

In [11]:
java_mods = df['diff'].apply(lambda x: sum([any('. java' in line for line in diff.split('<nl>')[:2]) for diff in x]))
print(f'% of .java modifications: {100 * java_mods.sum() / df["num_mods"].sum():.2f}%')

% of .java modifications: 87.16%


В некоторых примерах встречаются изменения вида `Binary files x and y differ`, которые не очень информативны. Посмотрим, какую долю от всех изменений составляют такие изменения:

In [12]:
binary_files_mods = df['diff'].apply(lambda x: sum(['Binary files' in diff for diff in x]))
print(f'% of "Binary files x and y differ" modifications: {100 * binary_files_mods.sum() / df["num_mods"].sum():.2f}%')

% of "Binary files x and y differ" modifications: 0.58%


### Немного примеров

In [13]:
idx = np.random.randint(len(df))
print(f'Diff ({df["diff_len"][idx]} tokens)', df['diff'][idx], sep='\n')
print()
print(f'Message ({df["message_len"][idx]} tokens)', df['message'][idx], sep='\n')
print()
print('Author', df['author'][idx], sep='\n')

Diff (103 tokens)
['<FILE> webanno - brat / src / main / java / de / tudarmstadt / ukp / clarin / webanno / brat / annotation / component / AnnotationDetailEditorPanel . java <nl> <nl> protected void onChange ( AjaxRequestTarget aTarget , ActionContext aBModel ) <nl> { <nl> - / / Overriden in BratAnnotator <nl> + / / Overriden in CurationPanel <nl> } <nl> <nl> protected void onAutoForward ( AjaxRequestTarget aTarget , ActionContext aBModel ) <nl> { <nl> - / / Overriden in BratAnnotator <nl> + / / Overriden in CurationPanel <nl> } <nl> <nl> protected void onAnnotate ( AjaxRequestTarget aTarget , ActionContext aModel ) <nl>']

Message (9 tokens)
 - Refactoring project structure <nl> - Correcting comment text

Author
('Richard Eckart de Castilho', 'richard.eckart@gmail.com')


## Сохраним данные в более удобном формате

In [15]:
# отсортируем датафрейм так, чтобы у каждого автора коммиты шли в соответствии с датой, от самых ранних к самым поздним
df.sort_values(by=['author', 'date'], inplace=True)

# заменим авторов на что-то вроде id, чтобы было проще
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['author'] = df['author'].apply(lambda x: x[0] + '<SEP>' + x[1]) # label encoder doesn't work with tuples
df['author'] = le.fit_transform(df['author'])

# сделаем из диффов-списков диффы-строки
df['diff'] = df['diff'].apply(lambda x: ' '.join(x))

Вынесем в train, val и test разные репозитории.

In [42]:
from sklearn.model_selection import train_test_split

repos = df['repo'].unique()

train_repos, test_repos = train_test_split(repos, test_size=0.01, shuffle=True, random_state=5)
train_repos, val_repos = train_test_split(train_repos, test_size=0.01, random_state=5)

In [43]:
print('Train:', len(train_repos))
print('Val:', len(val_repos))
print('Test:', len(test_repos))

Train: 975
Val: 10
Test: 10


In [44]:
test_df = df.loc[df['repo'].isin(test_repos)]
test_df.shape

(14185, 8)

In [45]:
val_df = df.loc[df['repo'].isin(val_repos)]
val_df.shape

(25230, 8)

In [46]:
train_df = df.loc[df['repo'].isin(train_repos)]
train_df.shape

(1559308, 8)

Теперь дополнительно проверим, что в train не встречаются авторы из val и test.

In [47]:
test_authors = test_df['author'].unique()
val_authors = val_df['author'].unique()

print('Intersection between val and test:')
print(np.intersect1d(test_authors, val_authors))

test_and_val_authors = list(test_authors) +  list(val_authors)

Intersection between val and test:
[28454]


In [48]:
train_df = train_df.loc[~train_df['author'].isin(test_and_val_authors)]
train_df.shape

(1520342, 8)

In [49]:
train_df.head()

,author,date,message,diff,num_mods,diff_len,message_len,repo
1390099,99,2014-05-11 21:19:07,And re - add renamed file . . .,<FILE> src / main / java / org / spongepowered...,3,235,9,spongepowered/spongeapi
1228073,100,2010-05-12 23:39:56,[ PAXWEB - 216 ] - added fail events,<FILE> pax - web - extender - war / src / main...,1,1574,9,ops4j/org.ops4j.pax.web
1228074,100,2010-06-12 00:32:02,[ PAXWEB - 218 ] - The next waiting webapp is ...,<FILE> pax - web - extender - war / src / main...,1,1103,12,ops4j/org.ops4j.pax.web
279027,101,2001-10-24 09:07:36,"Added method getText ( page , version ) , getH...",<FILE> src / com / ecyrd / jspwiki / WikiEngin...,1,451,42,apache/jspwiki
279088,101,2001-11-23 15:10:24,Initial version of tag library object for quer...,new file <nl> <FILE> src / com / ecyrd / jspwi...,1,1468,11,apache/jspwiki


In [50]:
train_df['author'].nunique()

56310

In [51]:
val_df['author'].nunique()

816

In [52]:
test_df['author'].nunique()

572

In [53]:
train_df[['diff', 'message', 'author']].to_csv('train.csv', header=None, index=None)
val_df[['diff', 'message', 'author']].to_csv('val.csv', header=None, index=None)
test_df[['diff', 'message', 'author']].to_csv('test.csv', header=None, index=None)